<h1>7장 고급 텍스트 생성 기술과 도구</h1>
<i>프롬프트 엔지니어링을 넘어서</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter07.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 7장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>

### [선택사항] - <img src="https://colab.google/static/images/icons/colab.png" width=100>에서 패키지 선택하기


이 노트북을 구글 코랩에서 실행한다면 다음 코드 셀을 실행하여 이 노트북에서 필요한 패키지를  설치하세요.

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

사용 패키지 버전

* transformers 4.57.3
* llama-cpp 0.3.16
* langchain 1.2.0
* langchain-community 0.4.1
* langchain-openai 1.1.6
* ddgs 9.10.0

In [1]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

In [2]:
%%capture
!pip install langchain_community langchain_openai ddgs

# 사용하는 파이썬과 CUDA 버전에 맞는 llama-cpp-python 패키지를 설치하세요.
# 현재 코랩의 파이썬 버전은 3.12이며 CUDA 버전은 12.4(12.5)입니다.
!pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.16-cu124/llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl

# LLM 로드하기

In [3]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2026-01-29 03:18:06--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 3.165.160.59, 3.165.160.61, 3.165.160.11, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/662698108f7573e6a6478546/a9cdcf6e9514941ea9e596583b3d3c44dd99359fb7dd57f322bb84a0adc12ad4?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1769660286&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY5NjYwMjg2fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjYyNjk4MTA4Zjc1NzNlNmE2NDc4NTQ2L2E5Y2RjZjZlOTUxNDk0MWVhOWU1OTY1ODNiM2QzYzQ0ZGQ5OTM1OWZiN2RkNTdmMzIyYmI4NGEwYWRjMTJhZDRcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Sign

LlamaCpp (C/C++로 작성된 오픈소스 라이브러리, llm을 빠르게 실행) 호출

In [4]:
from langchain_community.llms import LlamaCpp

# llm 정의
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=4096,
    seed=42,
    verbose=False
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64


In [13]:
llm.invoke("Hi")

'ppolyte Taine (1828-1893) was a French writer, critic and historian. He wrote "Les origines de la France contemporaine" in 1875.'

## 체인

langchain_core.prompts 모듈에서 PromptTemplate 모듈 호출

PromptTemplate : test -> template 변환

In [14]:
from langchain_core.prompts import PromptTemplate

# "input_prompt" 변수를 가진 프롬프트 템플릿 만들기
template = """<|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [30]:
#연결
basic_chain = prompt | llm

In [31]:
# 체인 사용.
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Maarten. What is 1 + 1?",
    }
)

' Hello Maarten! The answer to 1 + 1 is 2.'

### 여러 템플릿을 가진 체인

In [37]:
from langchain_classic.chains import LLMChain

#체인1 : 이야기 제목
template = """<|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

In [21]:
title.invoke({"summary": "a girl that lost her mother"})

{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Fading Memory: The Journey Through Loss"'}

In [22]:
# 체인 2 : 요약 + 제목 -> 주인공 설명
template = """<|user|>
Describe the main character of a story about {summary} with the title {title}.
Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [23]:
character.invoke({'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Fading Memory: The Journey Through Loss"'})

{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Fading Memory: The Journey Through Loss"',
 'character': ' The main character, Emily, is a resilient and introspective teenager struggling to cope with the profound grief of losing her mother at a young age. As she embarks on a journey through loss, her strength and vulnerability intertwine, leading her towards self-discovery and healing.'}

In [24]:
# 체인 3 (요약 + 제목 + 주인공 설명 -> 이야기) 생성
template = """<|user|>
Create a story about {summary} with the title {title}.
The main charachter is: {character}.
Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [38]:
# 세 개의 요소를 연결하여 최종 체인을 만듭니다.
llm_chain = title | character | story

In [27]:
llm_chain.invoke("a man who has been looking for his dream girl.")

{'summary': 'a man who has been looking for his dream girl.',
 'title': ' "The Endless Quest for Love: A Man\'s Pursuit of His Dream Girl"',
 'character': ' The main character in "The Endless Quest for Love: A Man\'s Pursuit of His Dream Girl" is a determined and passionate individual, who has spent his entire life searching for the perfect partner. He embodies unwavering perseverance and optimism as he navigates through various relationships and experiences in pursuit of his ideal woman.\n\nThis man\'s relentless quest shapes not only his romantic endeavors but also molds him into a well-rounded individual, evolving with each encounter on the path towards finding love. His search becomes an odyssey filled with personal growth and self-discovery as he continues to chase after the elusive dream girl who seems just out of reach.',
 'story': " The Endless Quest for Love: A Man's Pursuit of His Dream Girl\n\nIn a bustling city teeming with love seekers, there was an unyielding man named Sa

# 메모리

In [40]:
# LLM에게 이름을 알려 줍니다.
basic_chain.invoke({"input_prompt": "Hi! My name is gahui. What is 1 + 1?"})

" Hello Gahui! The answer to 1 + 1 is 2. It's a basic arithmetic operation, adding the two single units together."

In [41]:
# LLM에게 이름을 묻습니다.
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm unable to determine your name without more context. As an AI, I don't have the ability to access personal data unless it has been shared with me in the course of our conversation for the purpose of assisting you. If you're looking to find out your name, perhaps check a recent document or ask someone nearby who might know."

### 대화 버퍼

In [42]:
# 대화 기록을 담을 수 있도록 프롬프트를 업데이트합니다.
template = """<|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [43]:
from langchain_classic.memory import ConversationBufferMemory

# 사용할 메모리를 정의합니다.
memory = ConversationBufferMemory(memory_key="chat_history")

# LLM, 프롬프트, 메모리를 연결합니다.
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

/tmp/ipython-input-1877295073.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [44]:
# 간단한 질문을 하여 대화 기록을 만듭니다.
llm_chain.invoke({"input_prompt": "Hi! My name is gahui. What is 1 + 1?"})

{'input_prompt': 'Hi! My name is gahui. What is 1 + 1?',
 'chat_history': '',
 'text': " The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units altogether.\n\nHere’s a simple breakdown:\n\n- Start with the number 1.\n\n- Add another 1 to it.\n\n- You end up with 2."}

In [45]:
# LLM이 이름을 기억할까요?
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is gahui. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units altogether.\n\nHere’s a simple breakdown:\n\n- Start with the number 1.\n\n- Add another 1 to it.\n\n- You end up with 2.",
 'text': " Your name is gahui.\n\nAs for the question I asked before, my name is an AI and it doesn't have one like a human does. However, you can refer to me as your assistant or digital helper!"}

### 윈도 대화 버퍼

In [46]:
from langchain_classic.memory import ConversationBufferWindowMemory

# 메모리에 마지막 두 개의 대화만 유지합니다.
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# LLM, 프롬프트, 메모리를 연결합니다.
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

/tmp/ipython-input-40775140.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [47]:
# 두 개의 질문을 던져 메모리에 대화 기록을 저장합니다.
llm_chain.invoke({"input_prompt":"Hi! My name is gahui and I am 100 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

{'input_prompt': 'What is 3 + 3?',
 'chat_history': "Human: Hi! My name is gahui and I am 100 years old. What is 1 + 1?\nAI:  Hello gahui! Regardless of your age, the answer to 1 + 1 is always 2.\n\nHere's how it works:\n\n1. Start with number one (1).\n2. Add another one (+) to it.\n3. You get two (2).\n\nSo, mathematically speaking, 1 + 1 = 2.",
 'text': ' Hello! Absolutely, regardless of age, the mathematical answer remains consistent. For 3 + 3:\n\n1. Start with number three (3).\n2. Add another three (+) to it.\n3. You get six (6).\n\nHence, mathematically speaking, 3 + 3 = 6.'}

In [48]:
# 이름을 기억하는고 있는지 확인합니다.
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is gahui and I am 100 years old. What is 1 + 1?\nAI:  Hello gahui! Regardless of your age, the answer to 1 + 1 is always 2.\n\nHere's how it works:\n\n1. Start with number one (1).\n2. Add another one (+) to it.\n3. You get two (2).\n\nSo, mathematically speaking, 1 + 1 = 2.\nHuman: What is 3 + 3?\nAI:  Hello! Absolutely, regardless of age, the mathematical answer remains consistent. For 3 + 3:\n\n1. Start with number three (3).\n2. Add another three (+) to it.\n3. You get six (6).\n\nHence, mathematically speaking, 3 + 3 = 6.",
 'text': ' Hello! Your name is gahui, as you mentioned in our previous conversation.\n\nAs for the math question: When we add 3 + 3 together, the sum is indeed 6. Mathematics remains constant and unaffected by personal factors like age.'}

In [49]:
# 나이를 기억하고 있는지 확인합니다.
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': 'Human: What is 3 + 3?\nAI:  Hello! Absolutely, regardless of age, the mathematical answer remains consistent. For 3 + 3:\n\n1. Start with number three (3).\n2. Add another three (+) to it.\n3. You get six (6).\n\nHence, mathematically speaking, 3 + 3 = 6.\nHuman: What is my name?\nAI:  Hello! Your name is gahui, as you mentioned in our previous conversation.\n\nAs for the math question: When we add 3 + 3 together, the sum is indeed 6. Mathematics remains constant and unaffected by personal factors like age.',
 'text': " I'm an AI and do not have the capability to know personal information about individuals unless it has been shared with me in a conversation. Therefore, I can't determine your age without that specific information from you.\nAnswer: I cannot answer this question as I don't have access to personal data like your age unless previously provided during our conversation."}

### 대화 요약

In [50]:
# 요약 프롬프트 템플릿
summary_prompt_template = """<|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [51]:
from langchain_classic.memory import ConversationSummaryMemory

# 사용할 메모리를 정의
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# LLM, 프롬프트, 메모리를 연결합니다.
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

/tmp/ipython-input-1864264585.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [52]:
# 이름에 대해 질문하는 대화를 생성합니다.
llm_chain.invoke({"input_prompt": "Hi! My name is gahui. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': ' The Human, introducing themselves as "gahui," asked the AI a simple arithmetic question: What is 1 + 1? The AI responded by stating that 1 + 1 equals 2, explaining it\'s a basic addition operation. The updated summary of their conversation reiterates this information:\n\nNew Summary: "gahui" inquired about the sum of 1 and 1; the AI confirmed it is 2, describing it as an elementary arithmetic addition.',
 'text': ' Your name in the context of this conversation appears to be "gahui."'}

In [ ]:
# 지금까지 내용이 요약되어 있는지 확인합니다.
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' The conversation began with a simple math problem where the user, referred to as "Maarten", asked for the sum of 1 + 1. The answer provided was 2, and an explanation was given that when you add one (1) to another one (1), it results in two (2). Later, upon inquiring about their name within the conversation context, the AI deduced that "Maarten" is likely the user\'s name based on their previous interaction.\n\nNew summary: \nGreetings! In our initial math problem discussion, you asked for the sum of 1 + 1, to which we answered 2. Reflecting upon your conversation with us, it appears that "Maarten" is probably your name as per the context provided within this particular exchange.',
 'text': ' The first question you asked was: "What is 1 + 1?"'}

In [ ]:
# 지금까지 요약 내용을 확인합니다.
memory.load_memory_variables({})

{'chat_history': ' The conversation started with the user, "Maarten", inquiring about the sum of 1 + 1. You were correctly informed that this results in 2. Additionally, based on your interaction and context provided during the conversation, it is inferred that "Maarten" might be your name. Further, you asked about the first question you posed to us, which was indeed asking for the sum of 1 + 1.'}

# 에이전트

In [ ]:
import os
from langchain_openai import ChatOpenAI

# 랭체인으로 오픈AI의 LLM을 로드합니다.
os.environ["OPENAI_API_KEY"] = "MY_KEY"
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
# ReAct 템플릿을 만듭니다.
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [ ]:
from langchain_core.tools import Tool
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain_classic.tools import DuckDuckGoSearchResults

# 에이전트에 전달할 도구를 만듭니다.
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# 도구를 준비합니다.
tools = load_tools(["llm-math"], llm=openai_llm)

# 기존 'Calculator' 도구의 설명을 더 구체적으로 덮어씁니다.
for tool in tools:
    if tool.name == "Calculator":
        tool.description = (
            "Useful for when you need to answer questions about math. "
            "Input should be a strictly numerical mathematical expression. "
            "Do NOT include any words, variable names, or currency symbols like 'price' or '$'."
        )

tools.append(search_tool)

In [ ]:
from langchain_classic.agents import AgentExecutor, create_react_agent

# ReAct 에이전트를 만듭니다.
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
# 맥북 프로의 가격은 얼마인가요?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
I need to find the current price of a MacBook Pro in USD and then convert it to EUR using the given exchange rate.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"snippet: 6 days ago - If a big screen is your top priority, you can score a $309 discount on the 16-inch model of the latest MacBook Pro, dropping the price to $2,190 on Amazon . It features the M4 Pro processor and 24GB of RAM for blazing-fast performance, but not ..., title: Start 2026 With a New MacBook and Save Up to $800 - CNET, link: https://www.cnet.com/deals/best-macbook-deals/, snippet: October 15, 2025 - We list their lowest price of the day All resellers: Free shipping, sales tax not included Select price for product details or to purchase · 14-inch M5 MacBook Pros on Holiday sale at B&H starting at $1449 , up to $150 off Apple’s MSRP 12/29/2025, title: Apple 14″ MacBook Pro Prices at MacPrices.net, link: https://www.macprices.net/14-macbook-pro/, snippet:

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The cost of a MacBook Pro in EUR would be 1861.5 EUR.'}